In [1]:
import torch
from torchvision.transforms import transforms
from data import ZSSRdataset, ZSSRsampler
from utils import *
from model import ZSSRnet
from train import train, final_output
import numpy as np
from PIL import Image

In [2]:
num_batches = 3000
s_factor = 1.5
device = "cuda"

lres_img = Image.open('./images/01-lr.png')

In [3]:
dataset = ZSSRdataset.from_image(lres_img, s_factor)
data_sampler = ZSSRsampler(dataset)
dataset.__len__()

170

In [4]:
model = ZSSRnet()
model.to(device)

train(lres_img,
      model,
      data_sampler,
      num_batches,
      s_factor,
      device)

torch.save(model, './model/crop64/zssr1.pt')

Iteration: 300 Loss: 0.0015288980212062597, Learning Rate: 0.0001 :  10%|█         | 301/3000 [00:13<03:02, 14.77it/s]learning rate updated:  0.0001
Iteration: 600 Loss: 0.002755948342382908, Learning Rate: 1e-05 :  20%|██        | 601/3000 [00:25<02:42, 14.81it/s]learning rate updated:  1e-05
Iteration: 898 Loss: 0.0019719519186764956, Learning Rate: 1e-05 :  30%|██▉       | 899/3000 [00:37<01:28, 23.76it/s]learning rate updated:  1.0000000000000002e-06
Training is over.
Average Loss is 0.005070200789425479



In [5]:
hres_false = bicubic_resample(lres_img, s_factor)
hres_false.save('./final_result/crop64/zssr_local_01_200_gradual_x1.5_interpolated.png')

In [6]:
model = torch.load('./model/crop64/zssr1.pt')

final_zssr, outputs = final_output(lres_img, model, bicubic_resample, s_factor, back_projection)

final_zssr.save('./final_result/crop64/zssr_local_01_200_gradual_x1.5_net_out.png')

In [7]:
model = torch.load('./model/crop64/zssr1.pt')
num_batches = 3000
s_factor = 2 / 1.5
device = "cuda"

dataset2 = ZSSRdataset.from_image(final_zssr, s_factor)
data_sampler2 = ZSSRsampler(dataset2)

train(final_zssr,
      model,
      data_sampler2,
      num_batches,
      s_factor,
      device)
torch.save(model, './model/crop64/zssr1.pt')

Iteration: 299 Loss: 0.005348830483853817, Learning Rate: 0.0001 :  10%|█         | 300/3000 [00:14<01:56, 23.26it/s]learning rate updated:  0.0001
Iteration: 599 Loss: 0.0006533502019010484, Learning Rate: 1e-05 :  20%|██        | 600/3000 [00:28<01:43, 23.15it/s]learning rate updated:  1e-05
Iteration: 1138 Loss: 0.006439405493438244, Learning Rate: 1e-05 :  38%|███▊      | 1139/3000 [00:53<01:28, 21.12it/s]learning rate updated:  1.0000000000000002e-06
Training is over.
Average Loss is 0.003164476459865116



In [8]:
hres_false = bicubic_resample(final_zssr, s_factor)
hres_false.save('./final_result/crop64/zssr_local_01_200_gradual_x2_interpolated.png')

model = torch.load('./model/crop64/zssr1.pt')

final_zssr2, outputs2 = final_output(final_zssr, model, bicubic_resample, s_factor, back_projection)

final_zssr2.save('./final_result/crop64/zssr_local_01_200_gradual_x2_net_out.png')

In [9]:
direct_zssr = Image.open('zssr_local_01_200_x2_net_out.png')
direct_zssr.size

(422, 422)

In [10]:
final_zssr2.size

(421, 421)

In [11]:
direct_zssr = direct_zssr.resize([final_zssr2.size[0], final_zssr2.size[1]], resample=Image.BICUBIC)

In [12]:
direct_zssr_tensor = transforms.ToTensor()(direct_zssr)
final_zssr2_tensor = transforms.ToTensor()(final_zssr2)

residual = torch.abs(direct_zssr_tensor - final_zssr2_tensor)
residual_img = transforms.ToPILImage()(residual)
residual_img.save('./final_result/crop64/zssr_local_01_200_gradual_x2_residual.png')